In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
from concurrent.futures import ProcessPoolExecutor, as_completed


train_path = '/kaggle/input/fer2013/train/'
test_path = '/kaggle/input/fer2013/test/'


train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(124,124),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',
)


validation_generator = train_datagen.flow_from_directory(
    test_path,
    target_size=(124,124),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',
)


weight_decay = 1e-4
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(64, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay), input_shape=(124,124,1)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(128, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(128, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.GlobalAveragePooling2D())  # Add Global Average Pooling
model.add(tf.keras.layers.Dense(128, activation="linear"))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(7, activation='softmax'))


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])





input_train_folder = '/kaggle/input/fer2013/train'


def process_image(input_path):
    image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
    equalized_image = cv2.equalizeHist(image)
    cv2.imwrite(input_path, equalized_image)


def override_noise_reduction(input_folder, kernel_size=(5, 5)):
    for emotion_folder in os.listdir(input_folder):
        emotion_path = os.path.join(input_folder, emotion_folder)

        for filename in os.listdir(emotion_path):
            if filename.endswith('.jpg'):
                input_path = os.path.join(emotion_path, filename)

                
                image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
                blurred_image = cv2.GaussianBlur(image, kernel_size, 0)

                
                cv2.imwrite(input_path, blurred_image)

def override_histogram_equalization_parallel(input_folder, num_workers=4):
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = []

        for emotion_folder in os.listdir(input_folder):
            emotion_path = os.path.join(input_folder, emotion_folder)

            image_files = [filename for filename in os.listdir(emotion_path) if filename.endswith('.jpg')]

            for filename in image_files:
                input_path = os.path.join(emotion_path, filename)

                
                futures.append(executor.submit(process_image, input_path))

       
        for future in as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error loading the image : {e}")


override_noise_reduction(input_train_folder)

history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/30


2024-03-11 14:45:17.952780: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


898/898 [==============================] - 117s 123ms/step - loss: 1.7846 - accuracy: 0.2824 - val_loss: 1.8436 - val_accuracy: 0.2286
Epoch 2/30
898/898 [==============================] - 84s 93ms/step - loss: 1.5206 - accuracy: 0.4247 - val_loss: 1.5541 - val_accuracy: 0.3993
Epoch 3/30
898/898 [==============================] - 84s 93ms/step - loss: 1.3612 - accuracy: 0.5104 - val_loss: 1.4510 - val_accuracy: 0.4985
Epoch 4/30
898/898 [==============================] - 84s 93ms/step - loss: 1.2803 - accuracy: 0.5519 - val_loss: 1.3656 - val_accuracy: 0.5261
Epoch 5/30
898/898 [==============================] - 83s 93ms/step - loss: 1.2360 - accuracy: 0.5806 - val_loss: 1.4643 - val_accuracy: 0.4969
Epoch 6/30
898/898 [==============================] - 84s 93ms/step - loss: 1.2059 - accuracy: 0.5964 - val_loss: 1.2748 - val_accuracy: 0.5786
Epoch 7/30
898/898 [==============================] - 83s 93ms/step - loss: 1.1778 - accuracy: 0.6170 - val_loss: 1.3692 - val_accuracy: 0.5309
E

In [3]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
898/898 [==============================] - 82s 92ms/step - loss: 0.9618 - accuracy: 0.7612 - val_loss: 1.4806 - val_accuracy: 0.6113
Epoch 2/10
898/898 [==============================] - 82s 92ms/step - loss: 0.9498 - accuracy: 0.7645 - val_loss: 1.6129 - val_accuracy: 0.5897
Epoch 3/10
898/898 [==============================] - 83s 92ms/step - loss: 0.9533 - accuracy: 0.7643 - val_loss: 1.5562 - val_accuracy: 0.6144
Epoch 4/10
898/898 [==============================] - 83s 92ms/step - loss: 0.9572 - accuracy: 0.7689 - val_loss: 1.6505 - val_accuracy: 0.5823
Epoch 5/10
898/898 [==============================] - 82s 92ms/step - loss: 0.9502 - accuracy: 0.7691 - val_loss: 1.4568 - val_accuracy: 0.6300
Epoch 6/10
898/898 [==============================] - 82s 92ms/step - loss: 0.9423 - accuracy: 0.7736 - val_loss: 1.5846 - val_accuracy: 0.6198
Epoch 7/10
898/898 [==============================] - 83s 92ms/step - loss: 0.9501 - accuracy: 0.7772 - val_loss: 1.5169 - val_accuracy:

In [5]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
898/898 [==============================] - 86s 95ms/step - loss: 0.9352 - accuracy: 0.7851 - val_loss: 1.5618 - val_accuracy: 0.6151
Epoch 2/10
898/898 [==============================] - 85s 95ms/step - loss: 0.9297 - accuracy: 0.7864 - val_loss: 1.5080 - val_accuracy: 0.6252
Epoch 3/10
898/898 [==============================] - 85s 95ms/step - loss: 0.9286 - accuracy: 0.7895 - val_loss: 1.5351 - val_accuracy: 0.6330
Epoch 4/10
898/898 [==============================] - 83s 92ms/step - loss: 0.9285 - accuracy: 0.7906 - val_loss: 1.6193 - val_accuracy: 0.6053
Epoch 5/10
898/898 [==============================] - 83s 93ms/step - loss: 0.9281 - accuracy: 0.7903 - val_loss: 1.5491 - val_accuracy: 0.6266
Epoch 6/10
898/898 [==============================] - 83s 92ms/step - loss: 0.9167 - accuracy: 0.7965 - val_loss: 1.5437 - val_accuracy: 0.6174
Epoch 7/10
898/898 [==============================] - 83s 92ms/step - loss: 0.9174 - accuracy: 0.7971 - val_loss: 1.6057 - val_accuracy:

In [6]:
model.save('/kaggle/working/face_emotion.h5')